# CW + Diversity Regularization on CIFAR10

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision
import torchvision.transforms as transforms

import pickle
import datetime
import glob
import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('precision', 10)

# custom code imports
from neuron_coverage import *

import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

device = torch.device("cpu")
if torch.cuda.is_available():
    print('CUDA is available!')
    device = torch.device("cuda")
else:
    print('CUDA is not available...')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CUDA is available!


# Loading a Pretrained ResNet56
https://github.com/akamaster/pytorch_resnet_cifar10

In [2]:
from resnet import *

models_dir = 'pretrained_models/cifar10/' 
model = resnet56().cuda()
state_dict = torch.load(models_dir + 'resnet56.th', map_location='cuda')['state_dict'] # best_prec1, state_dict

new_state_dict = {}

for k, v in state_dict.items():
    if 'module' in k:
        k = k.replace('module.', '')
    new_state_dict[k]=v
    
model.load_state_dict(new_state_dict)

print('model loaded!')
# print(model)

model loaded!


# Load Data

In [3]:
data_dir = 'C:\data\CIFAR10'

classes = ['plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck']

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

batch_size = 100 # determines how many of each class we want

def get_same_index(targets, label):
    '''
    Returns indices corresponding to the target label
    which the dataloader uses to serve downstream.
    '''
    label_indices = []
    for i in range(len(targets)):
        if targets[i] == label:
            label_indices.append(i)
    return label_indices

dataset = torchvision.datasets.CIFAR10(root=data_dir, 
                                       train=False, 
                                       download=True,
                                       transform=transforms.Compose([
                                           transforms.ToTensor()
                                       ]))

Files already downloaded and verified


## Generate per class batches for NC evaluation

In [4]:
# data = []
# labels = []

# for i in range(len(classes)):
    
#     target_indices = get_same_index(dataset.targets, i)
    
#     test_loader = torch.utils.data.DataLoader(dataset,
#         batch_size=batch_size, 
#         sampler=torch.utils.data.sampler.SubsetRandomSampler(target_indices),
#         shuffle=False,
#         num_workers=2, 
#         pin_memory=True)
    
#     inputs, targets = next(iter(test_loader))
    
#     data.append(inputs)
#     labels.append(targets)
    
# # torch.Size([10, batch_size, 3, 32, 32])
# inputs = torch.stack(data).to(device)

# # torch.Size([new_batch_size, 3, 32, 32])
# new_batch_size = len(classes) * batch_size
# all_inputs = inputs.view(new_batch_size,3,32,32)

# # torch.Size([10, batch_size])
# targets = torch.stack(labels).to(device)

# # torch.Size([new_batch_size])
# all_targets = targets.view(-1)

# # individual class performance
# for i in range(len(classes)):
#     # confirm that loading the weights actually worked
#     orig_output = model(inputs[i])
#     orig_pred = torch.argmax(orig_output, dim=1)
#     orig_correct = orig_pred.eq(targets[i].data).sum()
#     orig_acc = 100. * orig_correct / len(targets[i])
#     print(classes[i] + '\t accuracy: {}/{} ({:.0f}%)'.format(orig_correct, len(targets[i]), orig_acc))

# # all class performance
# orig_output = model(all_inputs)
# orig_pred = torch.argmax(orig_output, dim=1)
# orig_correct = orig_pred.eq(all_labels.data).sum()
# orig_acc = 100. * orig_correct / len(all_labels)
# print('total accuracy: {}/{} ({:.0f}%)'.format(orig_correct, len(all_labels), orig_acc))

In [5]:
# # confirm that all the images are still intact after reshaping the tensor view
# orig_inputs = all_inputs.clone().cpu().detach().numpy()
# for img in orig_inputs:
#     plt.imshow(np.transpose(np.squeeze(img), (1, 2, 0))) 
#     plt.show()

## Evaluate per class and overall accuracy

In [31]:
def get_acc(model, device, test_loader, sampler=False, class_idx=None, classes=None):
    # model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
#             # peek at images to make sure they're of the right class
#             x = 0            
#             if x % 100 == 0:
#                 np_data = data.clone().cpu().detach().numpy()
#                 plt.imshow(np.transpose(np.squeeze(np_data[0]), (1, 2, 0))) 
#                 plt.show()               
#             x += 1
            
    if sampler:
        if class_idx is None:
            raise Exception('you must provide an integer class index if sampler=True')
        if classes is None:
            raise Exception('you must provide an iterable of class indices if sampler=True')
        values, counts = np.unique(test_loader.dataset.targets, return_counts=True) 
        divisor = counts[class_idx]
        acc = 100. * correct / divisor
        print(classes[class_idx] + '\t accuracy: {}/{} ({:.2f}%)'.format(correct, divisor, acc))
    else:
        divisor = len(test_loader.dataset)
        acc = 100. * correct / divisor
        print('accuracy: {}/{} ({:.2f}%)'.format(correct, divisor, acc))

In [35]:
# evaluate on total test set
test_loader = DataLoader(dataset,
    batch_size=batch_size, 
    shuffle=False,
    pin_memory=True)

get_acc(model, device, test_loader)

accuracy: 9107/10000 (91.07%)


In [36]:
# evaluate on each class separately
for i in range(len(classes)):
    
    target_indices = get_same_index(dataset.targets, i)
    
    test_loader = DataLoader(dataset,
        batch_size=batch_size,
        sampler=SubsetRandomSampler(target_indices),
        shuffle=False,
        pin_memory=True)
    
    get_acc(model, device, test_loader, sampler=True, class_idx=i, classes=classes)

plane	 accuracy: 215/1000 (21.50%)
car	 accuracy: 304/1000 (30.40%)
bird	 accuracy: 243/1000 (24.30%)
cat	 accuracy: 244/1000 (24.40%)
deer	 accuracy: 238/1000 (23.80%)
dog	 accuracy: 259/1000 (25.90%)
frog	 accuracy: 188/1000 (18.80%)
horse	 accuracy: 281/1000 (28.10%)
ship	 accuracy: 210/1000 (21.00%)
truck	 accuracy: 262/1000 (26.20%)


# Evaluating Neuron Coverage Per Class

In [16]:
nc_thresholds = np.arange(0,1,0.1)

neuron_coverages_by_class = {}

for i in range(len(classes)):
    
    threshold_results = {}
    
    for t in nc_thresholds:
        
        t = round(t,2)

        covered_neurons, total_neurons, neuron_coverage = eval_nc(model, inputs[i], t)
        print(classes[i] + ' neuron_coverage_' + str(t), neuron_coverage)

        threshold_results[str(t)] = neuron_coverage
    
    neuron_coverages_by_class[classes[i]] = threshold_results

plane neuron_coverage_0.0 0.8185937764089466


plane neuron_coverage_0.1 0.3672275535690811


plane neuron_coverage_0.2 0.20617100790625176


plane neuron_coverage_0.3 0.11765103570020094


plane neuron_coverage_0.4 0.06495521042648689


...

truck neuron_coverage_0.5 0.03556686510544799


truck neuron_coverage_0.6 0.01858063062217131


truck neuron_coverage_0.7 0.008452740896542658


truck neuron_coverage_0.8 0.002916486694585814


truck neuron_coverage_0.9 0.0007474318766549607


In [17]:
save_file_path = "assets/neuron_coverages_cifar10_10.pkl"
pickle.dump(neuron_coverages_by_class, open(save_file_path, "wb")) 

In [21]:
df = pd.DataFrame.from_dict(neuron_coverages_by_class)
df.to_clipboard(excel=True)

## Create Extreme Case Inputs

In [41]:
input_shape = (batch_size, 3, 32, 32)
all_blacks = torch.zeros(input_shape).to(device)
all_whites = torch.ones(input_shape).to(device)

In [42]:
covered_neurons, total_neurons, neuron_coverage = eval_nc(model, all_blacks, 0.0)
print('all_blacks neuron_coverage', neuron_coverage)

covered_neurons, total_neurons, neuron_coverage = eval_nc(model, all_whites, 0.0)
print('all_whites neuron_coverage', neuron_coverage)


all_blacks neuron_coverage 0.42952919303648895



all_whites neuron_coverage 0.439799808447107
